In [ ]:
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 
# !tar xvzf ta-lib-0.4.0-src.tar.gz
# import os
# os.chdir('ta-lib') # Can't use !cd in co-lab
# !./configure --prefix=/usr
# !make
# !make install
# os.chdir('../')
# !pip install TA-Lib

In [3]:
# !pip install ta
# !pip install pandasgui

  Created wheel for ta: filename=ta-0.8.0-py3-none-any.whl size=28895 sha256=f1598a501af7dfefc6e150edff7af2afead50899138a4a03a8cffe05a83fdd5f
  Stored in directory: c:\users\shiwa\appdata\local\pip\cache\wheels\a6\f0\22\5b8036f0825e6cab830da478c059b98aa60e44e84faf2099d0
Successfully built ta


In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
from ta.trend import ADXIndicator

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import talib

C:\Users\shiwa\Miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\shiwa\Miniconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\shiwa\Miniconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
Max_moving_avg=200
past_n_days=50

In [ ]:
# url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=reliance&apikey=UU42Y25QXPXJ319N'
# r = requests.get(url)
# data = r.json()

# print(data)

In [3]:

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=RELIANCE.BSE&outputsize=full&apikey=UU42Y25QXPXJ319N'
r = requests.get(url)
data = r.json()


In [19]:
stock_df=pd.DataFrame(data['Time Series (Daily)']).T.sort_index()

In [20]:
cols=['1. open', '2. high', '3. low', '4. close','6. volume', '7. dividend amount', '8. split coefficient']
stock_df[cols] = stock_df[cols].apply(pd.to_numeric, errors='coerce')

In [21]:
dates=(pd.to_datetime(stock_df.index))
stock_df['Is_prev_day_holiday']=[1 if((dates[i]-dates[i-1]).days>1) else 0 for i in range(len(dates)) ]

In [22]:
adxI = ADXIndicator(stock_df['2. high'],stock_df['3. low'],stock_df['4. close'],14,False)
stock_df['pos_directional_indicator'] = adxI.adx_pos()
stock_df['neg_directional_indicator'] = adxI.adx_neg()
stock_df['adx'] = adxI.adx()

C:\Users\shiwa\Miniconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\shiwa\Miniconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning:

invalid value encountered in double_scalars



In [23]:
stock_df['ADX_pos_neg_diff']=stock_df['pos_directional_indicator']-stock_df['neg_directional_indicator']
stock_df['ADX_pos_adx_diff']=stock_df['pos_directional_indicator']-stock_df['adx']
stock_df['ADX_neg_adx_diff']=stock_df['neg_directional_indicator']-stock_df['adx']
stock_df['ADX_pos_neg_diff']=(stock_df['ADX_pos_neg_diff'] - stock_df['ADX_pos_neg_diff'].mean()) / stock_df['ADX_pos_neg_diff'].std()
stock_df['ADX_pos_adx_diff']=(stock_df['ADX_pos_adx_diff'] - stock_df['ADX_pos_adx_diff'].mean()) / stock_df['ADX_pos_adx_diff'].std()
stock_df['ADX_neg_adx_diff']=(stock_df['ADX_neg_adx_diff'] - stock_df['ADX_neg_adx_diff'].mean()) / stock_df['ADX_neg_adx_diff'].std()
stock_df['Is_prev_day_holiday']=[1 if((dates[i]-dates[i-1]).days>1) else 0 for i in range(len(dates)) ]
stock_df['pos_directional_indicator'] = MinMaxScaler().fit_transform(np.array(stock_df['pos_directional_indicator']).reshape(-1,1))
stock_df['neg_directional_indicator'] = MinMaxScaler().fit_transform(np.array(stock_df['neg_directional_indicator']).reshape(-1,1))
stock_df['adx'] = MinMaxScaler().fit_transform(np.array(stock_df['adx']).reshape(-1,1))
stock_df['6. volume']=(stock_df['6. volume'] - np.mean(stock_df['6. volume'])) / np.std(stock_df['6. volume'])

In [24]:
stock_df['BBands_upper'], stock_df['BBands_mid'], stock_df['BBands_lower'] = talib.BBANDS(stock_df['4. close'], nbdevup=2, nbdevdn=2, timeperiod=20)
stock_df['RSI'] = talib.RSI(stock_df['4. close'], timeperiod=14)
stock_df['macd'], stock_df['macdsig_1'], stock_df['macdsig_2'] = talib.MACD(stock_df['4. close'], fastperiod=12, slowperiod=26, signalperiod=9)


Refreshed stock_df


In [10]:
from pandasgui import show

show(stock_df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [11]:
stock_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Is_prev_day_holiday,pos_directional_indicator,...,ADX_pos_neg_diff,ADX_pos_adx_diff,ADX_neg_adx_diff,BBands_upper,BBands_mid,BBands_lower,RSI,macd,macdsig_1,macdsig_2
2005-01-03,388.7685,407.0464,388.7685,405.9997,87.4919,0.014684,0.0,1.0,0,0.0,...,-0.106155,0.208378,0.287197,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,399.9444,407.3827,395.8329,396.8048,85.5104,0.012461,0.0,1.0,0,0.0,...,-0.106155,0.208378,0.287197,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,392.8427,399.1970,385.9277,396.1319,85.3654,0.021056,0.0,1.0,0,0.0,...,-0.106155,0.208378,0.287197,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,392.4690,401.0658,390.6747,393.1791,84.729,0.016683,0.0,1.0,0,0.0,...,-0.106155,0.208378,0.287197,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,394.7115,409.2889,394.3751,404.3176,87.1294,0.021075,0.0,1.0,0,0.0,...,-0.106155,0.208378,0.287197,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
Train_y=stock_df['4. close'].reset_index()[Max_moving_avg:]['4. close']

stock_df['Adjusted_close_price_13']=stock_df['4. close'].rolling(window=13).mean()
stock_df['Adjusted_close_price_50']=stock_df['4. close'].rolling(window=50).mean()
stock_df['Adjusted_close_price_100']=stock_df['4. close'].rolling(window=100).mean()
stock_df['Adjusted_close_price_200']=stock_df['4. close'].rolling(window=200).mean()
stock_df.columns

Index(['1. open', '2. high', '3. low', '4. close', '5. adjusted close',
       '6. volume', '7. dividend amount', '8. split coefficient',
       'Is_prev_day_holiday', 'pos_directional_indicator',
       'neg_directional_indicator', 'adx', 'ADX_pos_neg_diff',
       'ADX_pos_adx_diff', 'ADX_neg_adx_diff', 'BBands_upper', 'BBands_mid',
       'BBands_lower', 'RSI', 'macd', 'macdsig_1', 'macdsig_2',
       'Adjusted_close_price_13', 'Adjusted_close_price_50',
       'Adjusted_close_price_100', 'Adjusted_close_price_200'],
      dtype='object')

In [ ]:
# Train_y=stock_df['4. close'].reset_index()[Max_moving_avg:]['4. close']

In [28]:
column_to_Normalize=['1. open', '2. high', '3. low', '4. close',
                     'Adjusted_close_price_13', 'Adjusted_close_price_50','Adjusted_close_price_100', 'Adjusted_close_price_200',
                     'BBands_upper','BBands_mid','BBands_lower','pos_directional_indicator', 'neg_directional_indicator',
                     'adx', 'ADX_pos_neg_diff', 'ADX_pos_adx_diff', 'ADX_neg_adx_diff', 'RSI', 'macd', 'macdsig_1', 'macdsig_2']
column_to_exclude=['5. adjusted close']

In [ ]:
from sklearn import preprocessing
nr_scaler = preprocessing.StandardScaler()
X_train_normalized = nr_scaler.fit_transform(stock_df[Max_moving_avg:].reset_index()[column_to_Normalize])
col_not_normalized=[j for j in stock_df.columns if ((j not in column_to_Normalize) and (j not in column_to_exclude))]
print(col_not_normalized)
final_arr=[]

for i in range(len(X_train_normalized)):
    arr=list(X_train_normalized[i])
    arr.extend(list(stock_df.iloc[i][col_not_normalized]))
    final_arr.append(arr)

['6. volume', '7. dividend amount', '8. split coefficient', 'Is_prev_day_holiday']


In [ ]:
# Train_y

In [ ]:
scaler_for_y=MinMaxScaler()
Target=scaler_for_y.fit_transform(np.array(Train_y).reshape(-1,1))
Target


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:


train_X,Test_X,Train_y,Test_y=train_test_split(np.array(final_arr),Target,test_size=0.2,random_state=123,shuffle=False)

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
n_features=len(final_arr[0])
Train_generator=TimeseriesGenerator(train_X,Train_y,length=past_n_days,batch_size=1,sampling_rate=1)
Test_generator=TimeseriesGenerator(Test_X,Test_y,length=past_n_days,batch_size=1)

In [ ]:
# X,y=Train_generator[0]
# print(X.flatten())
# print(y)

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(past_n_days, n_features)))
# model.add(LSTM(150, return_sequences=True, input_shape=(Train_X.shape[1], 150)))
# model.add(LSTM(75, return_sequences=True, input_shape=(Train_X.shape[1], 150)))
model.add(LSTM(50, return_sequences=False, input_shape=(past_n_days, 75)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.summary()

In [ ]:
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
history = model.fit(Train_generator, epochs=55, callbacks=[callback])

In [ ]:
model.evaluate(Test_generator)

In [ ]:
Prediction=model.predict(Test_generator)

In [ ]:
from matplotlib import pyplot
# plot history
pyplot.plot(history.history['loss'], label='train')
# pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
len(Test_y[:,0][past_n_days:])
len(Prediction[:,0])

In [ ]:
Result_df=pd.DataFrame({'Actual':Test_y[:,0][past_n_days:],'predicted':Prediction[:,0] })

In [ ]:
Result_df['Actual']=scaler_for_y.inverse_transform(Test_y[:][past_n_days:])
Result_df['predicted']=scaler_for_y.inverse_transform(Prediction)

In [ ]:
from matplotlib import pyplot
# plot history
pyplot.plot(Result_df['Actual'], label='Actual')
pyplot.plot(Result_df['predicted'], label='predicted')
pyplot.legend()
pyplot.show()

In [ ]:
from datetime import datetime
now = datetime.now()
str(now)
model.save('saved_model/my_model'+str(now)) 

In [ ]:
# # Save the entire model as a SavedModel.
# !mkdir -p saved_model
# model.save('saved_model/my_model'+str(now)) 